In [30]:
import pandas as pd
import numpy as np
df=pd.read_csv("train.csv")
store=pd.read_csv("store.csv")
#df.head()
#store.head()
df_new=df.merge(store,on=["Store"],how="inner")
df_new['Date']=pd.to_datetime(df["Date"],infer_datetime_format=True)
df_new["Day"]=df_new["Date"].dt.day
df_new["Month"]=df_new["Date"].dt.month
df_new["Year"]=df_new["Date"].dt.year
df_new["Quarter"]=df_new["Date"].dt.quarter
#df_new.head()
#df_new.info()
#df_new.isnull().sum()/len(df_new)
df_new["CompetitionDistance"]=df_new["CompetitionDistance"].fillna(df_new["CompetitionDistance"].mode()[0])
df_new["CompetitionOpenSinceMonth"]=df_new["CompetitionOpenSinceMonth"].fillna(df_new["CompetitionOpenSinceMonth"].mode()[0])
df_new["CompetitionOpenSinceYear"]=df_new["CompetitionOpenSinceYear"].fillna(df_new["CompetitionOpenSinceYear"].mode()[0])
df_new["Promo2SinceWeek"]=df_new["Promo2SinceWeek"].fillna(df_new["Promo2SinceWeek"].mode()[0])
df_new["Promo2SinceYear"]=df_new["Promo2SinceYear"].fillna(df_new["Promo2SinceYear"].mode()[0])
df_new["PromoInterval"]=df_new["PromoInterval"].fillna(df_new["PromoInterval"].mode()[0])
df_new.isnull().sum()/len(df_new)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#df_new.info()
target=["Sales"]
categorical_columns=["DayOfWeek","Month","Year","StoreType","Assortment","Quarter"]
numerical_columns=["Customers","Open","Promo","Promo2","StateHoliday","SchoolHoliday","CompetitionDistance"]
def create_ohe(df, col):
    le = LabelEncoder()
    a = le.fit_transform(df[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse = False)
    column_names = [col + "_" + str(i) for i in le.classes_]
    return (pd.DataFrame(ohe.fit_transform(a), columns = 
                        column_names))

temp=df_new[numerical_columns]
for column in categorical_columns:
    temp_df=create_ohe(df_new,column)
    temp=pd.concat([temp,temp_df],axis=1)
#temp.info()
#temp["StateHoliday"].unique()
temp["StateHoliday"]=np.where(temp["StateHoliday"]=='0',0,1)
#temp.info()
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import History
history=History()
x_train,x_test,y_train,y_test=train_test_split(temp,df_new["Sales"],random_state=2019,test_size=0.2)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=2019,test_size=0.1)
#print(x_train.shape)
#print(y_train.shape)
#print(x_test.shape)
#print(y_test.shape)
#print(x_val.shape)
#print(y_val.shape)
model_1=Sequential()
model_1.add(Dense(150,input_dim=40,activation="relu"))
model_1.add(Dense(1,activation="linear"))
model_1.compile(optimizer='adam',loss="mean_absolute_error",metrics=["mean_absolute_error"])
model_1.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=64,callbacks=[history])

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categor

Train on 732390 samples, validate on 81377 samples
Epoch 1/10
732390/732390 [==============================] - 19s 26us/step - loss: 957.6309 - mean_absolute_error: 957.6309 - val_loss: 852.8037 - val_mean_absolute_error: 852.8037
Epoch 2/10
732390/732390 [==============================] - 19s 25us/step - loss: 797.3254 - mean_absolute_error: 797.3254 - val_loss: 790.2968 - val_mean_absolute_error: 790.2968
Epoch 3/10
732390/732390 [==============================] - 19s 26us/step - loss: 750.7812 - mean_absolute_error: 750.7812 - val_loss: 747.3588 - val_mean_absolute_error: 747.3588
Epoch 4/10
732390/732390 [==============================] - 19s 26us/step - loss: 734.6100 - mean_absolute_error: 734.6100 - val_loss: 729.2787 - val_mean_absolute_error: 729.2787
Epoch 5/10
732390/732390 [==============================] - 20s 27us/step - loss: 722.3947 - mean_absolute_error: 722.3947 - val_loss: 707.5096 - val_mean_absolute_error: 707.5096
Epoch 6/10
732390/732390 [=======================

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

NameError: name 'history' is not defined